In [3]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [5]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+'1'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')


In [6]:
trs = soup.select('form>table>tbody>tr')
len(trs)

100

In [7]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+'1'
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [9]:
len(driver.page_source), len(res.text)

(296712, 292114)

- Step 3

In [10]:
tr = trs[0]

In [11]:
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [12]:
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [13]:
channel = tr.select_one('.subject>h1>a').get_text().strip()
channel

'BLACKPINK'

In [14]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'8800만'

In [15]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view, video

('309억5913만', '514개')

In [16]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [17]:
convert('335억8260만')

33582600000

- Step 4

In [18]:
lines= []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject>h1>a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    lines.append({
        '순위':rank, '카테고리':category, '채널명':channel,
        '구독자수':subscriber, '조회수':view, '영상수':video
    })

In [19]:
df1 = pd.DataFrame(lines)
df1.tail()

,순위,카테고리,채널명,구독자수,조회수,영상수
95,96,키즈/어린이,뽀로로(Pororo),4910000,6878710000,4182
96,97,음식/요리/레시피,떵개떵,4890000,2990060000,4924
97,98,키즈/어린이,ToyMart TV,4860000,1994040000,695
98,99,애완/반려동물,SBS TV동물농장x애니멀봐,4840000,4654750000,4200
99,101,미분류,매일맛나 delicious day,4780000,488910000,328


100

In [8]:
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)
lines= []
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+ str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')
    
    
    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip()
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject>h1>a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({
            '순위':rank, '카테고리':category, '채널명':channel,
            '구독자수':subscriber, '조회수':view, '비디오수':video
        })
driver.close()

In [9]:
df = pd.DataFrame(lines)
df.shape


(1000, 6)

In [10]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오수
995,997,키즈/어린이,코코몽 COCOMONG TV - Cartoon & Song For Kids,730000,951630000,1420
996,996,취미/라이프,DanoTV,730000,158420000,642
997,1001,미분류,김지윤의 지식Play,730000,59060000,186
998,1000,미분류,성창경TV,730000,1036110000,16348
999,999,TV/방송,KBS여행 걸어서 세계속으로,730000,275530000,10959


In [11]:
df.to_csv('data/유튜브랭킹.csv',index=False)